<a href="https://colab.research.google.com/github/DIPANJAN001/Forecasting-Solar-Energy/blob/master/optimizer11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
!pip install Boruta
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from boruta import BorutaPy
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from keras.layers import concatenate
from keras import Model
from keras.layers import Input
from keras.layers import Bidirectional
from keras import layers
from sklearn.metrics import mean_squared_error
from sklearn.datasets import make_classification
from sklearn.datasets import make_classification
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from keras.layers import LeakyReLU
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import BatchNormalization
from keras.models import load_model
from keras.layers import Input
from sklearn.decomposition import PCA 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.6/56.6 KB 2.3 MB/s eta 0:00:00


In [2]:
def lstm_data_transform(x_data, y_data, num_steps):
    """ Changes data to the format for LSTM training 
for sliding window approach """
    # Prepare the list for the transformed data
    X, y = list(), list()
    # Loop of the entire data set
    for i in range(x_data.shape[0]):
        # compute a new (sliding window) index
        end_ix = i + num_steps
        # if index is larger than the size of the dataset, we stop
        if end_ix >= x_data.shape[0]:
            break
        # Get a sequence of data for x
        seq_X = x_data[i:end_ix]
        # Get only the last element of the sequency for y
        seq_y = y_data[end_ix]
        # Append the list with sequencies
        X.append(seq_X)
        y.append(seq_y)
    # Make final arrays
    x_array = np.array(X)
    y_array = np.array(y)
    return x_array, y_array

In [3]:
df=pd.read_excel("/content/pv_04.xlsx")
weather_input1=df.drop('power_normed',axis=1)
weather_input=weather_input1.drop('time_idx',axis=1)
solpow=df['power_normed']

In [14]:
from sklearn.feature_selection import RFE
from sklearn.linear_model import LinearRegression

# Define the model
model = LinearRegression()

# Define the feature selection method
rfe = RFE(model, n_features_to_select=15)

# Apply the feature selection method
X_train_ = rfe.fit_transform(weather_input,solpow)

In [15]:
X_train_[0]

array([1.00000000e+00, 3.73043347e-18, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 0.00000000e+00, 0.00000000e+00, 0.00000000e+00,
       0.00000000e+00, 7.03744759e-01, 5.25640098e-01, 9.68594902e-01,
       0.00000000e+00, 0.00000000e+00, 4.86973460e-04])

In [13]:
rfc = RandomForestRegressor(random_state=1, n_estimators=1000, max_depth=5)
boruta_selector = BorutaPy(rfc, n_estimators='auto', verbose=2, random_state=1)
boruta_selector.fit(X_train, np.array(solpow)) 

Iteration: 	1 / 100
Confirmed: 	0
Tentative: 	15
Rejected: 	0
Iteration: 	2 / 100
Confirmed: 	0
Tentative: 	15
Rejected: 	0
Iteration: 	3 / 100
Confirmed: 	0
Tentative: 	15
Rejected: 	0
Iteration: 	4 / 100
Confirmed: 	0
Tentative: 	15
Rejected: 	0
Iteration: 	5 / 100
Confirmed: 	0
Tentative: 	15
Rejected: 	0
Iteration: 	6 / 100
Confirmed: 	0
Tentative: 	15
Rejected: 	0
Iteration: 	7 / 100
Confirmed: 	0
Tentative: 	15
Rejected: 	0
Iteration: 	8 / 100
Confirmed: 	7
Tentative: 	4
Rejected: 	4
Iteration: 	9 / 100
Confirmed: 	7
Tentative: 	4
Rejected: 	4
Iteration: 	10 / 100
Confirmed: 	7
Tentative: 	4
Rejected: 	4
Iteration: 	11 / 100
Confirmed: 	7
Tentative: 	4
Rejected: 	4
Iteration: 	12 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	4
Iteration: 	13 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	4
Iteration: 	14 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	4
Iteration: 	15 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	4
Iteration: 	16 / 100
Confirmed: 	8
Tentative: 	3
Rejected: 	4
Iteration:

KeyboardInterrupt: ignored

In [ ]:
X_important_train = boruta_selector.transform(np.array(weather_input))
num_steps = 2
# training set
(x_transformed_train,
 y_transformed_train) = lstm_data_transform(X_important_train,solpow , num_steps=num_steps)
assert x_transformed_train.shape[0] == y_transformed_train.shape[0]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_transformed_train,y_transformed_train,test_size=0.33, random_state=42,shuffle=False)
#X_train_,X_val,y_train_,y_val=train_test_split(X_train,y_train,test_size=0.2, random_state=42,shuffle=False)
inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))

In [19]:
num_steps = 2

In [20]:
(x_transformed_train,
 y_transformed_train) = lstm_data_transform(X_train_,solpow , num_steps=num_steps)

In [21]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x_transformed_train,y_transformed_train,test_size=0.33, random_state=42,shuffle=False)

In [22]:
inputs1 = Input(shape=(X_train.shape[1],X_train.shape[2]))

In [23]:
inputs1

<KerasTensor: shape=(None, 2, 15) dtype=float32 (created by layer 'input_1')>

In [24]:
from keras import optimizers

In [25]:
from keras.optimizers import Adam
import numpy as np

class SPANDAN(Adam):
    def __init__(self, num_restarts=10, num_steps=100, lr=0.001, beta_1=0.9, beta_2=0.999,
                 epsilon=1e-8, decay=0., amsgrad=False, noise_std=0.1, **kwargs):
        super(SPANDAN, self).__init__(lr, beta_1, beta_2, epsilon, decay, amsgrad, **kwargs)
        self.num_restarts = num_restarts
        self.num_steps = num_steps
        self.noise_std = noise_std
        self.best_loss = np.inf
        self.best_weights = None
        self.initial_weights = None
    
    def get_weights(self):
        return self.best_weights

    def set_weights(self, weights):
        self.best_weights = weights
        self.initial_weights = weights

    def on_train_begin(self, logs=None):
        self.best_loss = np.inf
        self.initial_weights = self.model.get_weights()

    def on_batch_end(self, batch, logs=None):
        current_loss = logs.get('loss')
        if current_loss < self.best_loss:
            self.best_loss = current_loss
            self.best_weights = self.model.get_weights()

    def on_epoch_end(self, epoch, logs=None):
        if epoch % self.num_restarts == 0 and epoch > 0:
            for _ in range(self.num_steps):
                self.model.fit(self.model.train_on_batch, epochs=1, verbose=0)
                current_loss = logs.get('loss')
                if current_loss < self.best_loss:
                    self.best_loss = current_loss
                    self.best_weights = self.model.get_weights()
            # calculate distance 
            distance = [np.linalg.norm(b - i) for b, i in zip(self.best_weights, self.initial_weights)]
            distance = sum(distance)/len(distance)
            
            # Adding noise to the best parameters
            params_1 = [b + 2*distance for b in self.best_weights]
            self.model.set_weights(params_1)
            self.initial_weights = params_1

            # subtract noise to the best parameters*s
            params_2 = [b - 2*distance for b in self.best_weights]
            self.model.set_weights(params_2)
            self.initial_weights = params_2


In [26]:
from keras.layers import Bidirectional

In [ ]:
def define_model():



  '''fe1_0 = Bidirectional(LSTM(208, activation='relu',return_sequences = True))(inputs1)
  fe1_1 = Dropout(0.5)(fe1_0)
  fe1_2 = Bidirectional(LSTM(24, activation='relu',return_sequences = True))(fe1_1)
  fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=Bidirectional(LSTM(30, activation='relu'))(fe1_3)
  out1_1=Dense(1, activation='relu')(fe1_4)'''



  fe2_0 = Bidirectional(LSTM(208, activation='relu',return_sequences = True))(inputs1)
  fe2_1 = Dropout(0.6)(fe2_0)
  fe2_2 = Bidirectional(LSTM(60, activation='relu',return_sequences = True))(fe2_1)
  fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=Bidirectional(LSTM(4, activation='relu'))(fe2_3)
  out2_1=Dense(1, activation='relu')(fe2_4)

  fe3_0 =Bidirectional(LSTM(104, activation='relu',return_sequences = True))(inputs1)
  fe3_1 = Dropout(0.6)(fe3_0)
  fe3_2 = Bidirectional(LSTM(96, activation='relu',return_sequences = True))(fe3_1)
  fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=Bidirectional(LSTM(8, activation='relu'))(fe3_3)#16
  out3_1=Dense(1, activation='relu')(fe3_4)
 
 

  output = layers.average([ out3_1, out2_1])
  #merged3 = concatenate([out1_1,out2_1,out3_1], name='concat3')
  #output = Dense(1, activation='relu')( merged3)
  

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer =SPANDAN(),loss='mean_squared_error',metrics=['mse'])
 
  return model
mdl=define_model()
history=mdl.fit(X_train, y_train, epochs = 75,validation_split=0.2,batch_size =32)
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()
y=mdl.predict(X_test)
plt.scatter(y,y_test)
plt.show()
df = pd.DataFrame()
df['time']=[i for i in range(len(y))]
df['Actual']=y_test
df['Predicted']=y
plt.plot(df['time'],df['Actual'])
plt.plot(df['time'],df['Predicted'])
plt.show()
print(np.sqrt(mean_squared_error(y,y_test)))

Epoch 1/75
102/102 [==============================] - 25s 63ms/step - loss: 0.0123 - mse: 0.0123 - val_loss: 0.0031 - val_mse: 0.0031
Epoch 2/75
102/102 [==============================] - 4s 43ms/step - loss: 0.0055 - mse: 0.0055 - val_loss: 0.0023 - val_mse: 0.0023
Epoch 3/75
102/102 [==============================] - 4s 43ms/step - loss: 0.0047 - mse: 0.0047 - val_loss: 0.0020 - val_mse: 0.0020
Epoch 4/75
102/102 [==============================] - 4s 44ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0022 - val_mse: 0.0022
Epoch 5/75
102/102 [==============================] - 4s 43ms/step - loss: 0.0042 - mse: 0.0042 - val_loss: 0.0021 - val_mse: 0.0021
Epoch 6/75
102/102 [==============================] - 5s 49ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0019 - val_mse: 0.0019
Epoch 7/75
102/102 [==============================] - 4s 43ms/step - loss: 0.0038 - mse: 0.0038 - val_loss: 0.0017 - val_mse: 0.0017
Epoch 8/75
102/102 [==============================] - 5s 45ms/step -

In [ ]:
def define_model():


  # firs
  fe1_0 = Bidirectional(LSTM(208, activation='relu',return_sequences = True))(inputs1)
  fe1_1 = Dropout(0.5)(fe1_0)
  fe1_2 = Bidirectional(LSTM(24, activation='relu',return_sequences = True))(fe1_1)
  fe1_3= Dropout(0.2)(fe1_2)
  fe1_4=Bidirectional(LSTM(30, activation='relu'))(fe1_3)
  out1_1=Dense(1, activation='relu')(fe1_4)



  fe2_0 = Bidirectional(LSTM(208, activation='relu',return_sequences = True))(inputs1)
  fe2_1 = Dropout(0.6)(fe2_0)
  fe2_2 = Bidirectional(LSTM(60, activation='relu',return_sequences = True))(fe2_1)
  fe2_3= Dropout(0.5)(fe2_2)
  fe2_4=Bidirectional(LSTM(4, activation='relu'))(fe2_3)
  out2_1=Dense(1, activation='relu')(fe2_4)

  fe3_0 =Bidirectional(LSTM(104, activation='relu',return_sequences = True))(inputs1)
  fe3_1 = Dropout(0.6)(fe3_0)
  fe3_2 = Bidirectional(LSTM(96, activation='relu',return_sequences = True))(fe3_1)
  fe3_3= Dropout(0.5)(fe3_2)
  fe3_4=Bidirectional(LSTM(8, activation='relu'))(fe3_3)#16
  out3_1=Dense(1, activation='relu')(fe3_4)
 
 

  #output = layers.average([ out1_1, out2_1, out3_1])
  merged3 = concatenate([out1_1,out2_1,out3_1], name='concat3')
  output = Dense(1, activation='relu')( merged3)
  

  model = Model(inputs=[inputs1], outputs=[output])
  model.compile(optimizer=SPANDAN(),loss='mean_squared_error',metrics=['mse'])
 
  return model
mdl=define_model()
history=mdl.fit(X_train, y_train, epochs = 80,validation_split=0.2,batch_size =128)
import matplotlib.pyplot as plt
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='val')
plt.legend()
plt.show()
y=mdl.predict(X_test)
plt.scatter(y,y_test)
plt.show()
df = pd.DataFrame()
df['time']=[i for i in range(len(y))]
df['Actual']=y_test
df['Predicted']=y
plt.plot(df['time'],df['Actual'])
plt.plot(df['time'],df['Predicted'])
plt.show()
print(np.sqrt(mean_squared_error(y,y_test)))

Epoch 1/80
26/26 [==============================] - ETA: 0s - loss: 0.0218 - mse: 0.0218


KeyboardInterrupt



In [8]:
import numpy as np

In [11]:
a=np.random.random((1024,1024))
b=np.random.random((1024,1024))
c=a.dot(b)

In [12]:
c

array([[246.93599068, 251.28857176, 248.66852869, ..., 251.46631016,
        250.23583766, 251.76454704],
       [260.06509628, 261.93920332, 258.81997581, ..., 257.03646579,
        261.38082896, 261.77031216],
       [257.10204922, 255.02307827, 259.05375092, ..., 259.99902126,
        261.35000482, 260.29674168],
       ...,
       [262.55137229, 263.25127022, 267.01712133, ..., 266.08484901,
        263.39770393, 264.86754766],
       [261.68271347, 259.3241065 , 260.28616439, ..., 254.44014489,
        256.25114385, 260.25638259],
       [252.5376462 , 253.3472632 , 249.66980294, ..., 254.84257015,
        247.66169465, 254.69182449]])